<a href="https://colab.research.google.com/github/teju1001/textgen/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
ds= list()
def filetolist(txtfilelist) : 
    for txt_file in txtfilelist:
        with open(txt_fileencoding="utf8") as f:
            txt = f
            ds.append(txt)
    
filetolist(txt_files)
txtds=''.join(ds)       
print(len(txtds))

In [ ]:
import glob
txt_files= glob.glob("train/*.txt")
print(f"Total articles: {(txt_files[:10])}")

In [ ]:
import glob

def read_first_line(file):
    """Gets the first line from a file.

    Returns
    -------
    str
        the first line text of the input file
    """
    with open(file, encoding="utf8") as fd:
        first_line = fd.readline()
    return first_line

def merge_per_folder(folder_path, output_filename):
    """Merges first lines of text files in one folder, and
    writes combined lines into new output file

    Parameters
    ----------
    folder_path : str
        String representation of the folder path containing the text files.
    output_filename : str
        Name of the output file the merged lines will be written to.
    """
    # make sure there's a slash to the folder path 
    folder_path += "" if folder_path[-1] == "/" else "/"
    # get all text files
    txt_files = glob.glob(folder_path + "*.txt")
    # get first lines; map to each text file (sorted)
    output_strings = map(read_first_line, sorted(txt_files))
    output_content = "".join(output_strings)
    # write to file
    with open(folder_path + output_filename, 'wt') as outfile:
        outfile.write(output_content)

In [ ]:
merge_per_folder('testdata/', 'wikidata.txt')

In [ ]:
print(ds[1])

<_io.TextIOWrapper name='C:/Users/Dell/Documents/MiNi-PrOjEcT/train\\10.txt' mode='r' encoding='cp1252'>


# Code for Colab

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
import string
import requests
import numpy as np
import os
import time

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
text_file = open("/content/drive/My Drive/colab/wikidata2.txt",'r') #Drve File Path
ds= text_file.read() 
text_file.close()

In [4]:
ds[:500]

'सिकन्दरापुर मुंगेर मुंगेर बिहार स्थित एक गाँव है\n\nप्रियप्रवास अयोध्यासिंह हरिऔध की हिन्दी काव्य रचना है\nहरिऔध जी को काव्यप्रतिष्ठा प्रियप्रवास से मिली\nइसका रचनाकाल सन् से सन् है\nप्रियप्रवास विरहकाव्य है\n]काव्य की परंपरा में होते हुए भी उससे भिन्न है\nहरिऔध जी ने कहा है मैंने श्री कृष्णचंद्र को इस ग्रंथ में एक महापुरुष की भाँति अंकित किया है ब्रह्म करके नहीं\nकृष्णचरित को इस प्रकार अंकित किया है जिससे आधुनिक लोग भी सहमत हो सकें\nमहापुरुष के रूप में अंकित होते हुए भी प्रियप्रवास के कृष्ण में वही अलौक'

In [5]:
import re
def text_cleaner(text):
    newString = text
    #newString = re.sub(r"‘","",newString)
    # remove punctuations, english words and numbers
    newString = re.sub('([-%!#?,.:"‘’।])', " ", newString)
    newString = re.sub("[a-zA-Z]", " ", newString)
    newString = re.sub("[0-9]", " ", newString)
    long_words=[]
    # remove single characters
    for i in newString.split():
        if len(i)>=1:                  
            long_words.append(i)
    return (" ".join(long_words)).strip()

In [6]:
text = text_cleaner(ds).split()
print(text[:100])
print('\nTotal number of tokens :',len(set(text)))

['सिकन्दरापुर', 'मुंगेर', 'मुंगेर', 'बिहार', 'स्थित', 'एक', 'गाँव', 'है', 'प्रियप्रवास', 'अयोध्यासिंह', 'हरिऔध', 'की', 'हिन्दी', 'काव्य', 'रचना', 'है', 'हरिऔध', 'जी', 'को', 'काव्यप्रतिष्ठा', 'प्रियप्रवास', 'से', 'मिली', 'इसका', 'रचनाकाल', 'सन्', 'से', 'सन्', 'है', 'प्रियप्रवास', 'विरहकाव्य', 'है', ']काव्य', 'की', 'परंपरा', 'में', 'होते', 'हुए', 'भी', 'उससे', 'भिन्न', 'है', 'हरिऔध', 'जी', 'ने', 'कहा', 'है', 'मैंने', 'श्री', 'कृष्णचंद्र', 'को', 'इस', 'ग्रंथ', 'में', 'एक', 'महापुरुष', 'की', 'भाँति', 'अंकित', 'किया', 'है', 'ब्रह्म', 'करके', 'नहीं', 'कृष्णचरित', 'को', 'इस', 'प्रकार', 'अंकित', 'किया', 'है', 'जिससे', 'आधुनिक', 'लोग', 'भी', 'सहमत', 'हो', 'सकें', 'महापुरुष', 'के', 'रूप', 'में', 'अंकित', 'होते', 'हुए', 'भी', 'प्रियप्रवास', 'के', 'कृष्ण', 'में', 'वही', 'अलौकिक', 'स्फूर्ति', 'है', 'जो', 'अवतारी', 'ब्रह्मपुरुष', 'में', 'कवि', 'ने']

Total number of tokens : 287669


In [7]:
length = 30 + 1
lines =[]

for i in range (length,len(text)) :
    seq = text[i-length:i]
    line = ' '.join(seq)
    lines.append(line)
    if i >1100:
        break


In [8]:
lines[100]

'कृष्ण का चरित्रचित्रण मनोवैज्ञानिक दृष्टि से किया है उनके व्यक्तित्व में सहानुभूति व्युत्पन्नमतित्व और कर्मकौशल है कृष्ण के चरित्र की तरह प्रियप्रवास की राधा के चरित्र में भी नवीनता है उसमें'

In [9]:
## LSMT Model
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.utils import to_categorical 
from tensorflow. keras.models import Sequential 
from tensorflow.keras.layers import Dense, LSTM, Embedding 
from tensorflow.keras.preprocessing. sequence import pad_sequences

In [10]:
tokenizer = Tokenizer() 
tokenizer.fit_on_texts(lines) 
sequences = tokenizer.texts_to_sequences (lines)

In [11]:
sequences = np.array(sequences)
X,y = sequences[:,:-1],sequences[:,-1]

In [12]:
X[0]

array([478, 475, 475, 476,  41,  13, 474,   1,  11, 470,  81,   5, 469,
        86, 467,   1,  81,  85,  16, 465,  11,   7, 464,  83, 463,  84,
         7,  84,   1,  11])

In [13]:
y[0]

146

In [14]:
vocab_size = len(tokenizer.word_index)+1
vocab_size

479

In [15]:
y = to_categorical( y , num_classes = vocab_size)

In [16]:
X.shape[1]
seq_length = X.shape[1]

In [17]:
model = Sequential()
model.add(Embedding(vocab_size, 30 , input_length = seq_length))
model.add(LSTM(100 , return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100 , activation = 'relu'))
model.add(Dense (vocab_size , activation = 'softmax'))


In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 30, 30)            14370     
                                                                 
 lstm (LSTM)                 (None, 30, 100)           52400     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 479)               48379     
                                                                 
Total params: 205,649
Trainable params: 205,649
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.compile(loss = 'categorical_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'])

In [20]:
model.fit (X , y , batch_size =64 , epochs = 100)

Epoch 1/100
17/17 [==============================] - 6s 91ms/step - loss: 6.1384 - accuracy: 0.0308
Epoch 2/100
17/17 [==============================] - 1s 77ms/step - loss: 5.7397 - accuracy: 0.0420
Epoch 3/100
17/17 [==============================] - 1s 75ms/step - loss: 5.5405 - accuracy: 0.0598
Epoch 4/100
17/17 [==============================] - 1s 84ms/step - loss: 5.4940 - accuracy: 0.0598
Epoch 5/100
17/17 [==============================] - 1s 77ms/step - loss: 5.4777 - accuracy: 0.0598
Epoch 6/100
17/17 [==============================] - 1s 76ms/step - loss: 5.4717 - accuracy: 0.0598
Epoch 7/100
17/17 [==============================] - 1s 76ms/step - loss: 5.4718 - accuracy: 0.0598
Epoch 8/100
17/17 [==============================] - 1s 76ms/step - loss: 5.4695 - accuracy: 0.0598
Epoch 9/100
17/17 [==============================] - 1s 82ms/step - loss: 5.4658 - accuracy: 0.0598
Epoch 10/100
17/17 [==============================] - 1s 80ms/step - loss: 5.4627 - accuracy: 0.0598

In [32]:
seed_text='लेकिन जो एक'

In [22]:
def generate_text_sequence(model,tokenizer, text_seq_length , seed_text , n_words) :
    gtext =[]
    for _ in range(n_words):
        encoded = tokenizer.texts_to_sequences([seed_text])[0]
        encoded = pad_sequences([encoded], maxlen = text_seq_length , truncating = 'pre')
        predict_y=model.predict(encoded) 
        classes_y=np.argmax(predict_y,axis=1) 
        #y_predict = model.predict_classes(encoded)
        #print(y_predict)
        predicted_word = ' '
        for word , index in tokenizer.word_index.items():
            if index == classes_y:
                predicted_word = word
                break
        seed_text = seed_text + ' ' + predicted_word
        gtext.append(predicted_word)
    
    return ' ' .join(gtext)

In [33]:
x= generate_text_sequence(model, tokenizer, seq_length , seed_text , 20)
seed_text+ " " + x

'लेकिन जो एक युग बीत इसका मुहाना प्रमुख जलवायु हैं अपने दक्षिण में समशीतोष्ण शंकुधारी जंगलों और मध्य ब्राजील में ब्रजभाषा है भाषा'

In [24]:
seedt = 'उनके व्यक्तित्व'

In [25]:
x1 = seedt + ' ' + generate_text_sequence(model, tokenizer, seq_length , seedt , 20)
x1

'उनके व्यक्तित्व में किंतु एक युग बीत जाने में जाने में जाने में पिंडोरमा हो गई है जन्तुओं में प्रियप्रवास की संस्कृतबहुल'

In [29]:
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model

model.save('/content/drive/My Drive/colab/lstm.h5')
print('Model Saved!')
 

savedlstm=load_model('/content/drive/My Drive/colab/lstm.h5')
savedlstm.summary()

Model Saved!
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 30, 30)            14370     
                                                                 
 lstm (LSTM)                 (None, 30, 100)           52400     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 479)               48379     
                                                                 
Total params: 205,649
Trainable params: 205,649
Non-trainable params: 0
_________________________________________________________________


In [27]:
def gen_sent(savedlstm,tokenizer, text_seq_length , seed_text , n_words) :
    gtext =[]
    for _ in range(n_words):
        encoded = tokenizer.texts_to_sequences([seed_text])[0]
        encoded = pad_sequences([encoded], maxlen = text_seq_length , truncating = 'pre')
        predict_y=savedlstm.predict(encoded) 
        classes_y=np.argmax(predict_y,axis=1) 
        #y_predict = model.predict_classes(encoded)
        #print(y_predict)
        predicted_word = ' '
        for word , index in tokenizer.word_index.items():
            if index == classes_y:
                predicted_word = word
                break
        seed_text = seed_text + ' ' + predicted_word
        gtext.append(predicted_word)
    
    return ' ' .join(gtext)

In [28]:
xnew = seedt + ' ' + generate_text_sequence(model, tokenizer, seq_length , seedt , 20)
xnew

'उनके व्यक्तित्व में किंतु एक युग बीत जाने में जाने में जाने में पिंडोरमा हो गई है जन्तुओं में प्रियप्रवास की संस्कृतबहुल'